We can import the gui and instantiate a new session. If there is a saved session with the same name, it will be automatically loaded. To see the gui, use the `.draw()` method.

In [1]:
%matplotlib inline
from ironflow import GUI

In [2]:
gui = GUI('example')

/u/system/SLES12/soft/pyiron/dev/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: NOT-A-GIT-REPOSITORY is an invalid version and will not be supported in a future release
  warnings.warn(


Loaded session data for example


In [3]:
gui.draw()

We can also extend ironflow with new nodes on-the-fly. Most of the tools you should need are stored under `ironflow.custom_nodes`. Once we register a new node from a notebook, it immediately shows up under the `__main__` tab in the node selector.

In [83]:
from ironflow.custom_nodes import Node, NodeInputBP, NodeOutputBP, dtypes, input_widgets
import numpy as np
import matplotlib.pylab as plt
from pyiron_atomistics import Project


class My_Node(Node):
    title = "MyUserNode"
    init_inputs = [
        NodeInputBP(dtype=dtypes.Integer(default=1), label="foo")
    ]
    init_outputs = [
       NodeOutputBP(label="bar")
    ]
    color = 'cyan'

    def update_event(self, inp=-1):
        self.set_output_val(0, self.input(0) + 42)

        
class JobTable_Node(Node):
    title = "JobTable"
    init_inputs = [
        NodeInputBP(dtype=dtypes.Data(default=1), label="Project")
    ]
    init_outputs = [
       NodeOutputBP(label="Table")
    ]
    color = 'cyan'

    def update_event(self, inp=-1):
        self.set_output_val(0, self.input(0).job_table(all_columns=False))        
        
        
class PyironTable_Node(Node):
    title = "PyironTable"
    n_cols = 2   # TODO: allow user to change number of cols 
    
    init_inputs = [
        NodeInputBP(type_="exec", label="run"),
        NodeInputBP(type_="exec", label="remove"),
        NodeInputBP(dtype=dtypes.Data(size="m"), label="project"),
        NodeInputBP(dtype=dtypes.Char(default="table"), label="name")          
    ]
    for n in np.arange(n_cols):
        init_inputs.append(
            NodeInputBP(dtype=dtypes.Choice(default="get_job_name", items=["get_job_name"]), 
                        label=f"Col_{n+1}"
            )
        )
    init_outputs = [
        NodeOutputBP(label="Table")
    ]
    
    for n in np.arange(n_cols):
        init_outputs.append(NodeOutputBP(label=f"Col_{n+1}"))
        
    color = 'cyan'
    
    
    @property
    def _project(self):
        return self.input(2)

    @property
    def _name(self):
        return self.input(3)    
    
    def _run(self):
        self.table = self.input(2).create.table(self._name)
        self._job = self.table
        for n in np.arange(self.n_cols):
            getattr(self.table.add, self.inputs[n + 4].val)

        self.table.run()   

    def _remove(self):
        try:
            name = self._job.name  # Remove based on the run job, not the input name which might have changed...
            self._project.remove_job(name)
            self.set_output_val(0, None)
        except AttributeError:
            pass    
    
    def _update_column_choices(self):
        self.table = self.input(2).create.table()
        for n in np.arange(self.n_cols):
            col_input = self.inputs[n + 4]
            last_col = col_input.val
            available_cols = dir (self.table.add)

            if last_col not in available_cols:
                col_input.val = available_cols[0]
            col_input.dtype.items = available_cols

    def update_event(self, inp=-1):
        if inp == 0:
            self._run()
            self.exec_output(0)
        elif inp == 1:
            self._remove()
        elif inp > 1:
            self._update_column_choices()              
        
        df = self.table.get_dataframe()
        self.set_output_val(0, df)
        if len(df) > 0:
            for n in np.arange(self.n_cols):
                self.set_output_val(n+1, df.iloc[:, n+1])

                
class ProjectSelector_Node(Node):
    '''
    Filter and select projects related to a Master Job 
    '''
    title = "ProjectSelector"
    color = "#5d95de"
    
    init_inputs = [
        NodeInputBP(type_="exec", label="filter"),
        NodeInputBP(dtype=dtypes.Char(default="/cmmc/u"), label="RootPath"),  
        NodeInputBP(dtype=dtypes.Integer(default=0), label="Index") 
    ]
    
    init_outputs = [
        NodeOutputBP(label="Project"),
        NodeOutputBP(label="SelectedJobTable")
    ]    
    
    project = None
    
    def _filter(self):
        from pathlib import Path

        root_path = self.inputs[1].val
        pr = Project(root_path)
        df = pr.job_table(all_columns=True)
        print (np.unique(df.hamilton))
        self.df_murn = df[df.hamilton == 'Murnaghan']
        index = self.inputs[2].val
        p = Path(self.df_murn.projectpath.values[index]) / self.df_murn.project.values[index] / f'{self.df_murn.job.values[index]}_hdf5'
        self.project = Project(str(p))
    
    def update_event(self, inp=-1):
        self._filter()
        # if inp == 0:
        #     self._filter()
        #     self.exec_output(0)

        if self.project is not None:
            self.set_output_val(0, self.project)
            self.set_output_val(1, self.df_murn)
            
                    
class Plot2d_Node(Node):
    """
    A 2D matplotlib plot.

    Inputs:
        x pandas series: Data for the x-axis.
        y pandas series: Data for the y-axis.

    Outputs:
        fig (matplotlib.figure.Figure): The resulting figure after a `matplotlib.pyplot.plot` call on x and y.
    """

    title = "Plot2d"
    version = "v0.1"
    init_inputs = [
        NodeInputBP(dtype=dtypes.Data(size="m"), label="x"),
        NodeInputBP(dtype=dtypes.Data(size="m"), label="y"),
    ]
    init_outputs = [
        NodeOutputBP(type_="data", label="fig"),
    ]
    color = "#5d95de"

    def update_event(self, inp=-1):
        super().update_event()
        plt.ioff()
        fig = plt.figure()
        plt.clf()
        x = self.input(0)
        y = self.input(1)
        plt.plot(x.values, y.values)
        plt.xlabel(x.name)
        plt.ylabel(y.name)
        self.set_output_val(0, fig)
        plt.ion()        

If we save a session with a custom node, the same node needs to registered again *before* we load that session! To instantiate and load such a saved session all at once, extra node packages can be included using the optional `extra_node_packages` argument. This takes a `list` of node packages, which should either be a list of nodes that are children of `Node` (as in the example below) -- these appear under `__main__` in the gui, or a python module or path to a .py file. When registering nodes from a module or file, only those that inherit from `Node` *and* have a class name ending in `_Node` will be registered (this allows you to have your own node class templates and avoid loading the template itself by simply using regular python CamelCase naming conventions and avoiding ending in `_Node`). 

In [108]:
gui2 = GUI("example_with_custom_node", extra_nodes_packages=[[My_Node, JobTable_Node, PyironTable_Node, Plot2d_Node, ProjectSelector_Node]])

Loaded session data for example_with_custom_node


In [109]:
gui2.draw()

Note: When registering nodes from a module or file, they appear in the tab based on the end of the module/file path (excluding the .py convention). This is intentional since nodes from multiple sources may be conceptually linked, so they get grouped with every other node that has the same location terminus, but it's possible it could lead to naming conflicts. You're already able to override this with the underlying `GUI.register_nodes` method, which allows you to specify your own location using the optional `node_group` argument. In a future update we plan to provide the same capability when registering nodes at initialization.

In [110]:
pt = gui2.script.flow.nodes[2]
pt